In [31]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import degree
from torch_geometric.data import HeteroData
import dgl


In [56]:
num_items , num_users = 3,3
h_dim = 16
initi = nn.init.xavier_uniform_
emb_dict = nn.ParameterDict({"user":initi(nn.Parameter(torch.randn(size=(num_users,h_dim)))),"item":initi(nn.Parameter(torch.randn(size=(num_items,h_dim))))})
print(emb_dict["user"].shape)

torch.Size([3, 16])


In [32]:

g_dict = {
            ('user', 'likes', 'item'): ([0, 0, 2, 2],[ 0, 1, 2, 1]),
            ('item', 'liked_by', 'user'): ([0, 1, 2, 1], [0, 0, 2, 2]),
            ('user', 'follows', 'user'): ([0, 1, 2],[0, 1, 2])}

num_dict = {"user": 3, "item": 3}

g = dgl.heterograph(g_dict, num_nodes_dict=num_dict)



for srctype, etype, dsttype in g.canonical_etypes:
            src, dst = g.edges(etype=(srctype, etype, dsttype))
            dst_degree = g.in_degrees(
                dst, etype=(srctype, etype, dsttype)
            ).float()  # obtain degrees
            src_degree = g.out_degrees(
                src, etype=(srctype, etype, dsttype)
            ).float()
            norm = torch.pow(src_degree * dst_degree, -0.5)
            print(srctype, etype, dsttype)
            print(src_degree,dst_degree,norm)

item liked_by user
tensor([1., 2., 1., 2.]) tensor([2., 2., 2., 2.]) tensor([0.7071, 0.5000, 0.7071, 0.5000])
user follows user
tensor([1., 1., 1.]) tensor([1., 1., 1.]) tensor([1., 1., 1.])
user likes item
tensor([2., 2., 2., 2.]) tensor([1., 2., 1., 2.]) tensor([0.7071, 0.5000, 0.7071, 0.5000])


In [33]:
x = [1,2,2,1,3,0]
print(degree(torch.tensor(x),7))

tensor([1., 2., 2., 1., 0., 0., 0.])


In [34]:
def create_sample_graph_pyg():
    # Create a PyG heterogeneous graph
    data = HeteroData()

    # Add node features for 'user' and 'item' node types
    data['user'].x = torch.randint(low=1,high=9,size=(3, 5))  # 3 'user' nodes, feature size 5
    data['item'].x = torch.randint(low=10,high=19,size=(3, 5))  # 3 'item' nodes, feature size 5

    # Add self-loop edges for 'follows' relation (user-user self-loops)

    # Add user-item interaction edges for 'likes' relation (varied interactions)
    data['user', 'likes', 'item'].edge_index = torch.tensor([[0, 0, 2, 2], [0, 1, 2, 1]])

    # Add item-user interaction edges for 'liked_by' relation (reverse of 'likes')
    # data['item', 'liked_by', 'user'].edge_index = torch.tensor([[0, 1, 2, 1], [0, 0, 2, 2]])
    data['user', 'follows', 'user'].edge_index = torch.tensor([[0, 1, 2], [0, 1, 2]])  # Self-loops

    return data

graph = create_sample_graph_pyg()

In [35]:
from torch import Tensor


class NGCFLayer(MessagePassing):
    def __init__(self):
        super(NGCFLayer, self).__init__(aggr='add')  # Use 'add' aggregation
        

        

    def forward(self, graph:HeteroData):
        # Get node features
        feat_dict = {node_type:graph[node_type].x for node_type in graph.node_types}
        out_dict = {}
        edge_type  = graph.edge_types
        
        # Prepare message and update functions
        funcs = {}
        for srctype, etype, dsttype in edge_type:
            
            print("----------------------",(srctype, etype, dsttype))
            edge_index = graph[(srctype, etype, dsttype)].edge_index
            src,dst = edge_index
            print(edge_index)
            if srctype == dsttype:  # For self loops
                x = feat_dict[srctype]
                out = self.propagate(edge_index, x=x,rit="self_loop")
               
            else:
                norm = degree(dst,graph[dsttype].num_nodes)[dst]
                print(f"degree {dsttype}:",degree(dst,graph[dsttype].num_nodes)[dst])
                print(f"degree: {srctype}",degree(src,graph[srctype].num_nodes)[src])
                
                out = self.propagate(edge_index, x=(feat_dict[srctype],feat_dict[dsttype]),rit="cross",norm=norm.unsqueeze(1))
            
            if dsttype not in out_dict:
                out_dict[dsttype] = out
            else:
                out_dict[dsttype] += out

      
        return out_dict
    
    def message(self,x_j,x_i,rit,norm) -> Tensor:
        if rit =="self_loop":
            return x_j
        print(x_j.shape)
        print(x_j)
        print(norm.shape)
        return x_j*norm






In [36]:
print(graph)
print("users")
print(graph["user"].x)
print("items")
print(graph["item"].x)
print("edges")
print(graph["item"].num_nodes)
# print(graph[('item', 'liked_by', 'user')].edge_index)


HeteroData(
  user={ x=[3, 5] },
  item={ x=[3, 5] },
  (user, likes, item)={ edge_index=[2, 4] },
  (user, follows, user)={ edge_index=[2, 3] }
)
users
tensor([[3, 1, 5, 2, 4],
        [3, 7, 1, 6, 2],
        [8, 4, 7, 2, 5]])
items
tensor([[17, 14, 15, 17, 13],
        [16, 16, 17, 16, 11],
        [11, 11, 12, 10, 11]])
edges
3


In [37]:
model = NGCFLayer()
result = model(graph)
print(result)

---------------------- ('user', 'likes', 'item')
tensor([[0, 0, 2, 2],
        [0, 1, 2, 1]])
degree item: tensor([1., 2., 1., 2.])
degree: user tensor([2., 2., 2., 2.])
torch.Size([4, 5])
tensor([[3, 1, 5, 2, 4],
        [3, 1, 5, 2, 4],
        [8, 4, 7, 2, 5],
        [8, 4, 7, 2, 5]])
torch.Size([4, 1])
---------------------- ('user', 'follows', 'user')
tensor([[0, 1, 2],
        [0, 1, 2]])
{'item': tensor([[ 3.,  1.,  5.,  2.,  4.],
        [22., 10., 24.,  8., 18.],
        [ 8.,  4.,  7.,  2.,  5.]]), 'user': tensor([[3, 1, 5, 2, 4],
        [3, 7, 1, 6, 2],
        [8, 4, 7, 2, 5]])}
